# Tutorial 7

## Frequency and time synchronisation

In [ ]:
import numpy as np  # make the numpy package available and use 'np' as alias
import scipy.linalg as lin
import matplotlib.pyplot as plt  # plotting library
%matplotlib inline
YOUR_CODE_HERE = None  # placeholder

### Exercise 1
Carry out a data-based frequency synchronization of a PSK transmission!

*Note*: The time synchronization has already been carried out, the optimally samples received symbols are available.

#### a) Generate `N` equiprobable `M`-PSK symbols to be used as a reference sequence.

*Note*: First create the constellation diagram and then use `np.random.choice` to select symbols that are equally likely.

In [ ]:
M = 4  # constellation order
N = 100  # lenght of reference sequence

symbol_mapping = YOUR_CODE_HERE
ref_symbols = YOUR_CODE_HERE

In [ ]:
plt.scatter(ref_symbols.real, ref_symbols.imag)
plt.xlabel('I'); plt.ylabel('Q')
plt.title('Reference sequence symbols');

#### b) Simulate the interference by applying a frequency offset of 0.1 Hz and a random phase to the symbol stream.
*Note*: To do this, multiply `ref_seq` by a suitable complex exponential term.

In [ ]:
T = 1  # symbol duration in seconds
f_off = YOUR_CODE_HERE
phi_off = YOUR_CODE_HERE
symbols_with_offset = YOUR_CODE_HERE

#### c) Apart from the influences from b), the channel is an AWGN channel with an SNR of 20 dB. Create a suitable noise realization `noise` and apply it to the receive symbols` recv_symbols`.
*Note*: Think about how you can generate a complex normal distribution with `np.random.randn` and how it has to be scaled. Don't forget to convert the logarithmic representation of the SNR to linear! You can use `np.mean` and` np.var` to check the mean and variance.

In [ ]:
SNR_dB = 20
noise_variance_linear = YOUR_CODE_HERE
noise = YOUR_CODE_HERE
recv_symbols = YOUR_CODE_HERE

In [ ]:
plt.scatter(recv_symbols.real, recv_symbols.imag)
plt.xlabel('I'); plt.ylabel('Q')
plt.title('Received reference sequence symbols');

#### d)Estimate the frequency offset from the noisy received sequence! In the first step, multiply the received sequence point by point with the complex conjugate reference sequence before you determine the frequency offset from the phase.

*1. Note* : What happens to the phase of the data symbols during multiplication?

*2. Note* : Make sure you understand how frequency and phase are related!

In [ ]:
y = YOUR_CODE_HERE  # multiply the received signal with the complex conjugated reference sequence
estimated_f_off = YOUR_CODE_HERE

In [ ]:
print("Estimated frequency offset:", estimated_f_off, "Hz")
print("Estimation error:", abs(f_off - estimated_f_off) / f_off * 100, "%")

### e) Theory questions
1. Which parameter determines the range of correctable frequency offsets? 
2. Give the equation for the range of correctable frequency offsets! 
3. Why does a static phase offset in the received signal have no influence on the frequency estimation?
4. What is the disadvantage of using only one symbol for the duration of the entire sequence? 

### Exercise 2
Implement a feedback system with a Gardner detector for the synchronization of the sampling time!

*Note*: The frequency synchronization has already been carried out successfully.

#### a) Generate a random sequence of BPSK symbols of length `N` with an average power of 1!

In [ ]:
N = 1000
symbols = YOUR_CODE_HERE

In [ ]:
plt.stem(symbols)
plt.title("Random BPSK symbols");
plt.margins(0.1)

#### b) Use the given RRC pulse for pulse shaping! Create both transmitter and receiver filters!
*Note*: Neglect (by removing) the settling time in the end result in order to suppress edge effects!

In [ ]:
import rrc  # requires rrc.py to be accessible (e.g., in the same directory)
sps = 16  # samples per symbol
K = 8  # length of the impulse response in symbols
rho = 1  # RRC rolloff factor
g = rrc.get_rrc_ir(K * sps + 1, sps, 1, rho)  # RRC impulse response
g /= lin.norm(g)  # normalize the filter such that the sampled receive symbols have the same mean power

In [ ]:
rx_signal = YOUR_CODE_HERE  # baseband receive signal

In [ ]:
plt.plot(rx_signal)
plt.title("Received baseband signal")
plt.xlabel("Index"); plt.ylabel("Amplitude")
plt.margins(x=0, y=0.1)

#### c) Simulate a time shift by removing the first `n_off` samples.

In [ ]:
n_off = 5
r = YOUR_CODE_HERE

#### d) Complete the implementation of the timing algorithm by implementing the functions `TED` and` loop_filter`!
*Note 1*: The implementation closely follows the lecture (chapter synchronization, slides 42-55).

*Note 2*: Use the `rint` function where necessary to convert the float values of `tau` into integer values.

In [ ]:
def rint(x):
    return np.int(np.round(x))

class gardner_timing_recovery:
    e = [0]
    gamma = 1e-1
    tau = [0, 0]
    output_symbols = []
    
    def run(self, y):
        for k in range(y.size//sps - 1):
            self.output_symbols.append(y[k*sps + rint(self.tau[k])])
            if k > 0:
                self.e.append(self.TED(y, k))  # update error signal
                self.tau.append(self.loop_filter(k))
    
    def TED(self, y, k):
        return YOUR_CODE_HERE
    
    def loop_filter(self, k):
        return YOUR_CODE_HERE
    
    def rint(self, x):
        return int(round(x))
    
timing_sync = gardner_timing_recovery()
timing_sync.run(r)
r_sync = timing_sync.output_symbols

In [ ]:
plt.plot(timing_sync.e); plt.title("Error signal e"); plt.show();
plt.plot([rint(tau) for tau in timing_sync.tau]); plt.title("Estimated timing offset tau"); plt.ylim([-sps//2-1, sps//2+1]); plt.show();
plt.stem(r[::sps]); plt.title("Unsynchronized receive symbols"); plt.show();
plt.stem(r_sync); plt.title("Synchronized receive symbols"); plt.show();

#### e)  Theory questions
1. What is the meaning of the parameter `gamma`? What is the trade-off in his choice?
2. Can you also use this algorithm to correct time offsets that change over time?
3. The procedure fails if `n_off = sps // 2`. Explain why! Is this a realistic case? Which other influences were neglected in this simulation? 